# Finding Pinnacle Partition by Length

In [ ]:
from itertools import permutations
from scipy.signal import find_peaks
from collections import Counter

## Peak to Permutation:

In [ ]:
#Creates list of tuples, [(peak,permutation),...], for a given symmetric group.
def loopName(seriesNumber):
    seriesList = []
    for i in range(1,seriesNumber+1):
        seriesList.append(i)

    vals = list(permutations(seriesList))
    tempList = []

    for val in vals:
        num = find_peaks(val)[0].tolist()
        tempList.append(([x+1 for x in num],list(val)))
    return tempList

In [ ]:
#Previous "peak set with unique partitions" grouping function from distributionSequence.py
class sumFunction:
    def __init__(self,cls,seriesNumber):
        self.currResult = []
        self.finalResult = []
        self.seriesList = []

        for i in range(1,seriesNumber+1):
            self.seriesList.append(i)

        cls.algorithm(self,cls,seriesNumber)

    def algorithm(self,cls,seriesNumber):
        cls.FirstVals(self,seriesNumber)
        cls.RemainVals(self,cls,seriesNumber)
        self.finalResult = [[]] + self.finalResult
        print(len(self.finalResult))
        return self.finalResult
            
    def FirstVals(self,seriesNumber):
        if seriesNumber%2 == 0:
            for i in range(1,int(seriesNumber/2)):
                tempList1 = []
                tempList2 = []
                tempList1.append(self.seriesList[i])
                tempList2.append(self.seriesList[-i-1])
                self.currResult.append((tempList1,tempList2))
        else:
            for i in range(1,int((seriesNumber+1)/2)):    
                tempList1 = []
                tempList2 = []
                tempList1.append(self.seriesList[i])
                tempList2.append(self.seriesList[-i-1])
                if tempList1 == tempList2:
                    self.currResult.append(tempList1)
                else:
                    self.currResult.append((tempList1,tempList2))  
        
        self.finalResult += self.currResult

    def RemainVals(self,cls,seriesNumber):     
        usedList = []
        for val in self.currResult:
            if isinstance(val,tuple):
                for j in range(2,seriesNumber-val[0][-1]):
                    tempList = []
                    tempVal1 = val[0]+[val[0][-1]+j]
                    tempVal2 = [val[1][0]-j] + val[1]
                    if tempVal1 == tempVal2:
                        tempList.append(tempVal1)
                        if len(tempVal1) >= (seriesNumber-2)/2:
                            self.finalResult += tempList
                            return
                        else:                            
                            self.finalResult += tempList
                            self.currResult += tempList
                            break
                    else:
                        if len(tempVal1) > (seriesNumber-2)/2:
                            return
                        elif tempVal1 in usedList:                           
                            break
                        tempList.append((tempVal1,tempVal2))
                        usedList.append(tempVal2)
                        self.finalResult += tempList
                        self.currResult += tempList

In [ ]:
symmetricGroupNum = 6  ##### Enter Symmetric Group Here

peakAndPerm=loopName(symmetricGroupNum)
print(peakAndPerm)

result = sumFunction(sumFunction,symmetricGroupNum)
print(result.finalResult)

## Peak to Pinnacle Mapping:

In [ ]:
#Creates list of tuples, [(peak,pinnacle),...], from loopName function output.
pinnacle=[]
for val in peakAndPerm:
    temp=[]
    for i in val[0]:
        temp.append(val[1][i-1])
    pinnacle.append((tuple(val[0]),tuple(sorted(temp))))
    
print(pinnacle)

In [ ]:
#Counts duplicate (peak -> pinnacle) mappings.
print(Counter(pinnacle))

## Pinnacle to Length:

In [ ]:
#Creates list of tuples, [(pinnacle,length),...], from loopName function output.
pinnacleLen=[]
for val in peakAndPerm:
    temp=[]
    for i in val[0]:
        temp.append(val[1][i-1])
    pinnacleLen.append((tuple(sorted(temp)),Permutation(val[1]).length()))
    
print(pinnacleLen)

In [ ]:
#Counts duplicate (pinnacle -> length) mappings, of the form ((pinnacle,length),count)
sorted(list(Counter(pinnacleLen).items()),key=lambda x:x[0])

#### Symmetry Checker:

In [ ]:
def symmetryChecker(pinLenList):
    counter = 0
    for i in range(1,len(pinLenList)):
        if pinLenList[i-1][0][0] == pinLenList[i][0][0]:
            counter += 1
        else:
            if counter%2==0:
                for j in range(counter/2):
                    print(pinLenList[i-1-counter+j][1],pinLenList[i-1-j][1])
                    if pinLenList[i-1-counter+j][1] != pinLenList[i-1-j][1]:
                        return False
            else:
                for j in range((counter-1)/2):
                    print(pinLenList[i-1-counter+j][1],pinLenList[i-1-j][1])
                    if pinLenList[i-1-counter+j][1] != pinLenList[i-1-j][1]:
                        return False      
            counter = 0
        if i == len(pinLenList)-1:          
            if counter%2==0:
                for j in range(counter/2):
                    print(pinLenList[i-counter+j][1],pinLenList[i-j][1])
                    if pinLenList[i-counter+j][1] != pinLenList[i-j][1]:
                        return False,1,counter,i,j
            else:
                for j in range((counter+1)/2):
                    print(pinLenList[i-counter+j][1],pinLenList[i-j][1])
                    if pinLenList[i-counter+j][1] != pinLenList[i-j][1]:
                        return False,2,counter,i,j
    return True

In [ ]:
symmetryChecker(pinLenList)

## Descent to Length:

In [ ]:
#Creates list of tuples, [(descent,length),...], from loopName function output.
descentLen=[]
for val in peakAndPerm:
    temp=[]
    for i in val[0]:
        temp.append(val[1][i-1])
    descentLen.append((tuple(Permutation(val[1]).descents()),Permutation(val[1]).length()))
    
print(descentLen)

In [ ]:
#Counts duplicate (descent -> length) mappings, of the form ((descent,length),count)
sorted(list(Counter(descentLen).items()),key=lambda x:x[0])

## Peak Inversions:

In [ ]:
# For non-symmetric peak sets, corresponding peak set tuple = PS
# Creates dictionary of the form: 
# {(PS[0],PS[1]) : [ (PS[0] permutation, (PS[0] lehmer, PS[1] inverse reverse lehmer)) ,...]}

peakDict={}
for peak in result.finalResult:
    if isinstance(peak,tuple):
        temp=[]
        for val in peakAndPerm:
            if val[0]==peak[0]:
                temp.append((val[1],
                             (Permutation(val[1]).to_lehmer_code(),
                              Permutation(Permutation(val[1][::-1]).inverse()[::-1]).to_lehmer_code())))
        peakDict.update({(tuple(peak[0]),tuple(peak[1])):temp})
                
print(peakDict)